In [151]:
import numpy as np
from math import floor
from math import ceil
from math import atan
from math import pi
import cv2 as cv


"""
	The Functions perpendicular(perp) and segment
	intersect are used actually to compute 
	the intersection of the detected line segments with
	the quads
"""

def perp(a) :
    b = np.empty_like(a)
    b[0] = -a[1]
    b[1] = a[0]
    return b

def seg_intersect(a1,a2,b1,b2) :
    da = a2-a1
    db = b2-b1
    dp = a1-b1
    dap = perp(da)
    denom = np.dot( dap, db)
    num = np.dot( dap, dp )
    return (num / denom.astype(float))*db + b1


"""
Reading the corresponding file generated by the Line Segment Detector
"""


def quantize_and_get(X,Y,threshold,linesegthreshold,dx,dy,delta):

	"""
		Preprocessing --- Reading the file corresponding to the image
		that was generated using the Line Segment Detector that the
		authors have used.
	"""

	file = open('/home/deepak/Desktop/Vision Project/Content Aware Rotation/Codes/fig1.png.txt','r')
	lines_list = []
	for i in file:
	    arr = i.split()
	    for elem in range(len(arr)):
	        arr[elem] = float(arr[elem])
	    lines_list.append(arr)
	file.close()

	"""
		This variable lines will store the lines in the following format
		(x1,y1,x2,y2,Orientation Angle, Bin Number)
	"""

	lines = np.zeros((1,6))
	lines_count = 0

	"""
		Counting the Number of Line Segments 
	"""

	for i in range(len(lines_list)):
	    
	    """
	    	Initial Checking for how the line is present.
	    	We're effectively checking for positions of 
	    	the corresponding points
	    """

	    x1 = lines_list[i][0]; y1 = lines_list[i][1]; x2 = lines_list[i][2]; y2 = lines_list[i][3]
	    if(max(x1,x2)>X or max(y1,y2)>Y or min(x1,x2,y1,y2)<1):
	        continue
	    if((x1-x2)**2 + (y1-y2)**2)<linesegthreshold:
	        continue
	    if x1>x2:
	        right_x = x1
	        left_x = x2
	    else:
	        left_x = x1
	        right_x = x2
	    if y1>y2:
	        y_top = y2
	        y_bottom = y1
	    else:
	        y_top = y1
	        y_bottom = y2

	    xi = np.array([x1,x2])
	    yi = np.array([y1,y2])

	    
	    
	    xi_ = np.array([x1,x2])
	    yi_ = np.array([y1,y2])

	    """
	    	Compute the intersection between the line segment and the 
	    	grid lines that are parallel to the Y-Axis
	    """

	    value = ceil(left_x/dx)*dx
	    while(value<=floor((right_x)/dx)*dx):
	        
	        arg1,arg2 = np.array([x1,y1]),np.array([x2,y2])
	        val = seg_intersect(arg1,arg2,np.array([value,0]),np.array([value,Y]))
	        #print(val,arg1,arg2)
	        if val[0]!=np.inf and val[0]!=np.nan and val[1]!=np.inf and val[1]!=np.nan: 
	            xi_ = np.append(xi_,val[0])
	            yi_ = np.append(yi_,val[1])
	        value = value+dx

	    """
	    	Compute the intersection between the line segment and 
	    	the grid lines parallel to the X-Axis
	    """
	    
	    value = ceil(y_top/dy)*dy
	    while(value<=floor(y_bottom/dy)*dy+1):
	        arg1,arg2 = np.array([x1,y1]),np.array([x2,y2])
	        val = seg_intersect(arg1,arg2,np.array([0,value]),np.array([X,value]))
	        if val[0]!=np.inf and val[0]!=np.nan and val[1]!=np.inf and val[1]!=np.nan :
	            xi_ = np.append(xi_,val[0])
	            yi_ = np.append(yi_,val[1])
	        value = value+dy
	        
	    xi_sorted = []
	    yi_sorted = []
	    for i_ in range(len(xi_)):
	        xi_sorted.append(xi_[i_])
	        yi_sorted.append(yi_[i_])
	    
	    
	    val_ = np.argsort(yi_)
	    yi_sorted.sort()
	    new_arr = []
	    for i_ in range(len(val_)):
	        new_arr.append(xi_sorted[val_[i_]])
	    xi_sorted = new_arr
	    
	    """
	    	Assigning the orientations to the lines, after sorting
	    	them in order to be able to quantize them so that
	    	each line segment belongs to only one quad.
	    """

	    for j in range(len(yi_sorted)-1):
	        if((xi_sorted[j]-xi_sorted[j+1])**2+(yi_sorted[j]-yi_sorted[j+1])**2 < linesegthreshold):
	            continue
	        else:
	            lines_count+=1

	            temp = np.zeros((1,6))
	            temp[0][0] = xi_sorted[j]
	            temp[0][1] = yi_sorted[j]
	            temp[0][2] = xi_sorted[j+1]
	            temp[0][3] = yi_sorted[j+1]
	            temp[0][4] = np.arctan((temp[0][3]-temp[0][1])/(temp[0][2]-temp[0][0]))*180/pi
	            if temp[0][4]<=0:
	                temp[0][4]+=180
	            temp[0][5] = ceil((temp[0][4]+delta)/(180/90))
	            if temp[0][5]<0:
	                temp[0][5] = temp[0][5]+90
	            if temp[0][5]>90:
	                temp[0][5] = temp[0][5]-90
	            lines = np.append(lines,temp)
	"""
		Reshaping the Lines to the requiste format
	"""
	lines = lines[6:]
	lines = lines.reshape((lines_count,6))

	return lines

In [152]:
import numpy as np
import pandas as pd

"""
    Some Notes in this Code - You are using 1 Indexed Co-Ordinates
    Need a way to fix them up sometime later.
    @9th November 2018 Update
"""

def formboundary(N,X,Y,gridX,gridY):
    """
        energy_vx = energy for x coordinate vertices
        energy_vy = energy for y coordinate vertices
        energy_v = overall total energy
        b = the RHS in the first part of the Optimization that we'll
        be doing
    """

    energy_vx = np.zeros(N)
    energy_vy = np.zeros(N);
    energy_v = np.zeros(2*N)
    bx = np.zeros(N)
    by = np.zeros(N)
    b = np.zeros(2*N)

    """
        Initialization of all the variables as in above
    """

    tempgridX = gridX.astype(np.uint16)
    tempgridY = np.rint(gridY)

    """
        We need to construct matrices that capture the boundary vertices under consideration
    """
    
    idx = (tempgridX == 0)
    idx = idx.transpose().reshape(N)
    energy_vx[idx]=1
    bx[idx] = -1

    """
        First the Left Border was covered above.
    """

    idx = (tempgridX==X-1)
    idx = idx.transpose().reshape(N)
    energy_vx[idx] = 1
    bx[idx] = -(X)
    """
        Right Border was covered above
    """

    idx = (tempgridY==0)
    idx = idx.transpose().reshape(N)
    energy_vy[idx]=1
    by[idx] = -(1)

    """
        Top border was covered above.
    """

    idx = (tempgridY == Y-1)
    idx = idx.transpose().reshape(N)
    energy_vy[idx] = 1
    by[idx] = -(Y)

    """
        Bottom Border was covered above
    """
    

    for i in range(N):
        energy_v[2*i] = energy_vx[i]
        energy_v[2*i+1] = energy_vy[i]
        b[2*i] = bx[i]
        b[2*i+1] = by[i]

    """
        Finally, I have computed the required energy function for
        all the vertices and made a matrix out of it.
    """

    boundary_matrix = np.diag(energy_v)
    return boundary_matrix,b

In [153]:
import numpy as np
from math import floor, ceil
"""
	This is a module to compute Pk; the breakdown of the eK's in the paper
	into Pk times V. 
"""

"""
	x,y are the same as vertexX and vertexY => len(x) =cx and len(y) = cy;
	N is the number of vertices
	dx, dy are the increments that we use across the grids.
	Again 1 Indexed - The 1st row is full of zeros
"""
def computepk(lines,dx,dy,N,x,y):

	Pk = np.zeros((len(lines),N))
	# PkV = np.zeros(len(lines),N)
	# Pk1 = np.zeros(len(lines),N)
	# Pk2 = np.zeros(len(lines),N)
	x = x+1
	y = y+1
	for i in range(len(lines)):

		x1 = lines[i][0]+1
		y1 = lines[i][1]+1
		x2 = lines[i][2]+1
		y2 = lines[i][3]+1
		
		min_x = min(x1,x2) 
		min_y = min(y1,y2)
		"""
			This is for extracting the corresponding Quad to be
			found for Bilinear Interpolation
		"""
		xleft = floor((min_x-1)/dx)+1
		xright= xleft+1

		ytop = floor((min_y-1)/dy)+1
		ybottom= ytop+1

		"""
			Initialising the matrices for bilinear interpolation.
			We'll subtracting these matrices and then constructing the needed matrix
		"""

		Pkmatrix1 = np.zeros((len(y),len(x)))
		Pkmatrix2 = np.zeros((len(y),len(x)))
		PkmatrixV = np.zeros((len(y),len(x)))

		Pkmatrix1[ytop-1,xleft-1] = ((x[xright-1]-x1)/dx)*((y[ybottom-1]-y1)/dy)
		Pkmatrix1[ytop-1,xright-1]= ((x1-x[xleft-1])/dx)*((y[ybottom-1]-y1)/dy)
		Pkmatrix1[ybottom-1,xleft-1]= ((x[xright-1]-x1)/dx)*((y1-y[ytop-1])/dy)
		Pkmatrix1[ybottom-1,xright-1]=((x1-x[xleft-1])/dx)*((y1-y[ytop-1])/dy)

		Pkmatrix2[ytop-1,xleft-1] = ((x[xright-1]-x2)/dx)*((y[ybottom-1]-y2)/dy)
		Pkmatrix2[ytop-1,xright-1]= ((x2-x[xleft-1])/dx)*((y[ybottom-1]-y2)/dy)
		Pkmatrix2[ybottom-1,xleft-1]= ((x[xright-1]-x2)/dx)*((y2-y[ytop-1])/dy)
		Pkmatrix2[ybottom-1,xright-1]=((x2-x[xleft-1])/dx)*((y2-y[ytop-1])/dy)

		Pk[i] = np.reshape((Pkmatrix2- Pkmatrix1).transpose(),N)
		
	return Pk

In [255]:
import numpy as np
import math
import pandas as pd

"""
	Incomplete as on 10th November! Bad - This is horrible!
"""

def getuk(lines):
	uk = np.zeros((len(lines),2))
	for i in range(len(lines)):
		x =  lines[i][2] - lines[i][0]
		y = lines[i][3] - lines[i][1]
		temp = np.array([x,y])
		try:
			temp = temp/np.linalg.norm(temp)
			uk[i] = temp
		except:
			temp = np.array([0,0])
			uk[i] = temp
	return uk


def formline(lines,number_of_vertices,dx,dy,x,y,thetas):
	U_final = np.zeros((len(lines),2,2))
	"""
		This is to form the Energy Function for 
		adding the constraint to preserve the line segments.
		This is a very important part of the project.
	"""
	#UAll = np.zeros(((2,2),len(lines)))
	uk = getuk(lines)
	uk = np.matrix(uk)
	df = pd.DataFrame(uk)
	N = number_of_vertices
	Pk = computepk(lines,dx,dy,N,x,y)
	matrix = np.zeros((2*N,2*N))
	tempPk1 = np.zeros((2*N))
	tempPk2 = np.zeros((2*N))
	Pk_ = np.zeros(((2*len(lines)),2*N))
	
	for i in range(len(lines)):
		Pk_final = np.zeros((2,2*N))
		for j in range(N):
			tempPk1[2*j] = Pk[i][j]
			tempPk2[2*j+1] = Pk[i][j]
		df = pd.DataFrame(tempPk1)
		Pk_final[0] = tempPk1
		Pk_final[1] = tempPk2
		theta = thetas[int(lines[i][5]) - 1]*np.pi/180
		U = uk[i].transpose().dot(uk[i])
		val = (np.array(uk[i]).dot(np.transpose(np.array(uk[i]))))
		U = np.array(U)*val
		U_final[i] = U
		Rk = np.array([[math.cos(theta),-math.sin(theta)],[math.sin(theta),math.cos(theta)]])
		temp = (Rk.dot(U).dot(np.transpose(Rk)) - np.eye(2)).dot(Pk_final)
		inter = np.transpose(temp).dot(temp)
		matrix+=inter
		Pk_[2*(i)] = Pk_final[0]
		Pk_[2*i+1] = Pk_final[1]
	df = pd.DataFrame(Pk_)
	return Pk_,matrix,U_final

In [256]:
import numpy as np
import pandas as pd
"""
	This function computes the value of StS, see report for further details
	First Row is full of zeros
"""

def formshape(x,y,number_of_vertices,quad_count):
	global aq1    # X and Y are VertexX and Vertexy
	N = number_of_vertices
	matrix = np.zeros((2*N,2*N))
	x = x+1
	y = y+1
	for i in range(len(y)):
		for j in range(len(x)):
			""" Each Quad is taken care of. (i,j) = Refers to the 
				top left vertex of every quad
			"""
			try:
				Aq = np.asarray([[x[j],-y[i],1,0],
								[y[i],x[j],0,1],
								[x[j],-y[i+1],1,0],
								[y[i+1],x[j],0,1],
								[x[j+1],-y[i],1,0],
								[y[i],x[j+1],0,1],
								[x[j+1],-y[i+1],1,0],
								[y[i+1],x[j+1],0,1]])
				
				#print(Q)
				
				# Decompose Vq as Q times V. 
				# Dont care about V for the time being
				# |x| = |y| = Number of Vertices making up the grid.
				# Non Trivial Decomposition
			
				Q = np.zeros((8,2*N))
				Q[0][2*((j)*(len(y))+i+1)-2] = 1
				Q[1][2*((j)*(len(y))+i+1)-1] = 1
				Q[2][2*((j)*(len(y))+i+1)] = 1
				Q[3][2*((j)*(len(y))+i+1)+1] = 1
				Q[4][2*((j+1)*(len(y))+i+1)-2] = 1
				Q[5][2*((j+1)*(len(y))+i+1)-1] = 1
				Q[6][2*((j+1)*(len(y))+i+1)] = 1
				Q[7][2*((j+1)*(len(y))+i+1)+1] = 1
				
				temp_matrix = (Aq.dot(np.linalg.pinv(Aq.transpose().dot(Aq))).dot(Aq.transpose()) - np.eye(8)).dot(Q)
				#print(Q)#print(np.linalg.pinv(Aq.transpose().dot(Aq)))
				matrix+=temp_matrix.transpose().dot(temp_matrix); 
				
                
				"""print(Aq)
																print(np.transpose(Aq))
																for i in range(len(Aq)):
																	for j in range(len(Aq[0])):
																		print(np.transpose(Aq)[i,:].dot(Aq[:,j]))
																break"""
			except:
				continue
				
				
				
		
	df = pd.DataFrame(matrix)
	return matrix

In [510]:
""" 

Content Aware Rotation, In ICCV 2013	
Authors: Kaiming He, Huiwen Chan, Jian Sun

An implementation by S Deepak Narayanan, Indian Institute of Technology Gandhinagar

"""


""" 
Standard Imports for the rest of the program 
"""
import pandas as pd
import matplotlib.pyplot as plt
import cv2 as cv
import numpy as np 
import random as rd 
"""from energyfunction import total_energy
from optimization import *
from line_matrix import *
from shape_matrix import *
from extractlines import quantize_and_get
from boundary_matrix import *
from pk import *"""

"""
Here I am initialising all the parameters
"""

rotation_angle = 6.1
img = cv.imread('/home/deepak/Desktop/Vision Project/Content Aware Rotation/Codes/fig1.png')
Y,X = img.shape[:2]

"""
	Total Number of Quads created = 900. This is very 
	deterimental in deciding the amount of time the 
	program takes to run
"""

"""
	cx = Number of Quads along the horizontal direction
	cy = Number of Quads along the vertical direction
	n_quads = Total number of quads
	number_of_vertices  = Total Number of Mesh Vertices
	x_len = Distance between subsequent Quads along x_direction
	y_len = Distance between subsequent Quads along y_direction
	threshold and linesegthreshold = Used for choosing the correct 
		Line Segements among those that were detected.
	vertexX = List of all the X_coordinates of the grid points
	vertexY = List of all the Y_coordinates of the grid points
	gridX,gridY =  Meshgrid that we form using these vertices
"""

cx= int(X/30)
cy = int(Y/30)

n_quads = cx*cy
number_of_vertices = (cx+1)*(cy+1)
x_len = (X-1)/cx
y_len = (Y-1)/cy


threshold = 16
linesegthreshold = (x_len**2 + y_len**2)/64

temp = 0
vertexX = np.zeros(1)
while(1):
    temp+=x_len
    temp = (round(temp,10))
    vertexX = np.append(vertexX,temp)
    if temp>X-2:
        break
temp = 0
vertexY = np.zeros(1)
while(1):
    temp+=y_len
    temp = (round(temp,10))
    vertexY = np.append(vertexY,temp)
    if temp>Y-2:
        break
gridX, gridY = np.meshgrid(vertexX,vertexY)
Vx = np.reshape(gridX,number_of_vertices,1)
Vy = np.reshape(gridY,number_of_vertices,1)
V = np.zeros((number_of_vertices*2))
for i in range(number_of_vertices):
    V[2*i] = Vx[i]
    V[2*i+1] = Vy[i]
V = V+1
print(X,Y)
delta = 6.1

sdelta = np.zeros(90)
sdelta[0] = 1000
sdelta[44] = 1000
sdelta[45] = 1000
sdelta[89] = 1000
print("Line Extraction and Quantization Begin....")
lines = quantize_and_get(X,Y,threshold,linesegthreshold,x_len,y_len,delta)
print("Line Extraction and Quantization Done .... ")
print("Forming the functions for Shape, Line, Boundary and Rotation Constraints...")

#print(gridX)

"""
    Intialising Thetas Now
"""

thetas = np.ones((90))*delta

Pk_all,line,UK = formline(lines,number_of_vertices,x_len,y_len,vertexX,vertexY,thetas)
shape_preservation = formshape(vertexX,vertexY,number_of_vertices,n_quads)
boundary,b = formboundary(number_of_vertices,X,Y,gridX,gridY)

lambda_l = 100
lambda_b = 10**8
lambda_r = 100  

"""
	Optimization Begin
"""
print('Boundary Size is ',boundary.shape)
print('Shape Size is ',shape_preservation.shape)
print('Line Size is ',line.shape)
print('PK_all is ',Pk_all.shape)
print('b is ',b.shape)

n = number_of_vertices
k = len(lines)
print(n,k)
V_new = np.zeros(len(V))

for number_of_iteration in range(1,11):
	print("Iteration Number ", number_of_iteration)
	V_new = fix_theta_solve_v(line,shape_preservation,boundary,b,lambda_l,lambda_r,lambda_b,n,k)
	print("V Distance is ")
	print(np.linalg.norm(V-V_new)**2)
	thetas = fix_v_solve_theta(UK,lines,thetas,V_new,rotation_angle,Pk,dx,dy,N,x,y,sdelta)
	Pk_all,line,UK = formline(lines,number_of_vertices,x_len,y_len,vertexX,vertexY,thetas)

899 674
Line Extraction and Quantization Begin....
Line Extraction and Quantization Done .... 
Forming the functions for Shape, Line, Boundary and Rotation Constraints...
Boundary Size is  (1380, 1380)
Shape Size is  (1380, 1380)
Line Size is  (1380, 1380)
PK_all is  (2626, 1380)
b is  (1380,)
690 1313
Iteration Number  1
Solving the sparse linear system of equations now... - Theta is a constant; Solving for V
V Distance is 
295679.24473078706
Iteration Number  2
Solving the sparse linear system of equations now... - Theta is a constant; Solving for V
V Distance is 
290733.2099953778
Iteration Number  3
Solving the sparse linear system of equations now... - Theta is a constant; Solving for V
V Distance is 
286895.4727365634
Iteration Number  4
Solving the sparse linear system of equations now... - Theta is a constant; Solving for V
V Distance is 
285021.8938201904
Iteration Number  5
Solving the sparse linear system of equations now... - Theta is a constant; Solving for V
V Distance is

In [511]:
thetas

array([6.09177494, 5.34592849, 5.338824  , 5.44388149, 5.45083934,
       5.45737731, 5.43413614, 5.45521537, 5.46650068, 5.477786  ,
       5.45000819, 5.43677631, 5.43032489, 5.48052776, 5.48447301,
       5.47270473, 5.45064953, 5.42859433, 5.432088  , 5.4306385 ,
       5.42981319, 5.43155428, 5.4318775 , 5.48702149, 5.48924464,
       5.48580348, 5.50823089, 5.53065829, 5.54032708, 5.54342446,
       5.5426924 , 5.59637962, 5.62293692, 5.64949422, 5.69318692,
       5.70656645, 5.75106358, 5.76460319, 5.82719755, 5.91624066,
       5.89580005, 5.89713584, 5.92974796, 5.98459505, 6.09092416,
       6.08993277, 5.97893195, 5.93110954, 5.89085973, 5.89193729,
       5.8909664 , 5.90338373, 5.8168139 , 5.86788329, 5.81342889,
       5.75899132, 5.73164808, 5.70430485, 5.70425524, 5.70590289,
       5.75587229, 5.7067109 , 5.67875943, 5.65080796, 5.69866765,
       5.65056006, 5.6532725 , 5.65610593, 5.65298063, 5.64985533,
       5.64929487, 5.67677556, 5.70425625, 5.72101582, 5.73777

In [514]:
import matplotlib.pyplot as plt
import pylab
"""
    This file is used to draw the grid. We will initially draw 
    the grid consisting of about 400 square quads. Then later on
    we will show the warped mesh, following which, we will show the 
    actually warped image.
"""


def warpmesh(image,X,Y):
    final = np.zeros(image.shape,Vx,Vy)
    """
        This is to warp the image. We shall use a barycentric coordinate
        based approach to perform the same. 
    """
    lineX = np.linspace(1,1,X)
    lineY = np.linspace(1,1,Y)
    numX = len(lineX)
    numY = len(lineY)
    gridX = np.ones(numY).dot(lineX)
    gridY = lineY.dot(np.ones(numX))
    sampleX = gridX.transpose().reshape((numX*numY))
    sampleY = gridY.transpose().reshape((numX*numY))
    sampleXY = np.zeros((2,sampleX.shape[1]))
    ori_sampleXY = np.zeros(sampleXY.shape)
    tmpI = np.zeros((3,sampleXY.shape[1]))
    count = np.zeros((3,samplyXY.shape[2]))

    for i in range(cx):
        for j in range(cy):
            v4 = np.array([i*(cy+1)+j+1,i*(cy+1)+j+2,(i+1)*(cy+1)+j+1,(i+1)*(cy+1)+j])
            for k in range(2):
                if k ==0:
                    tri_v = np.array([v4[0],v4[1],v4[3]])
                else:
                    tri_v = np.array([v4[0],v4[2],v4[3]])
                cornerX = Vx[tri_v]
                cornerY = Vy[tri_v]
                cornerXY = np.append(cornerX, cornerY, axis = 0)
                d_sampleXY = sampleXY - cornerXY[:,2].dot(np.ones((sampleXY,2)))
                T = cornerXY[:,0:2] - cornerXY[:,2].dot(np.ones((1,2)))
                lamb = np.linalg.inv(T).dot(d_sampleXY)
                index_table = (lamb[1,:]<=1)
                

899

899

In [507]:
import numpy as np 
import matplotlib.pyplot as plt 
import math

def fix_theta_solve_v(line,shape,boundary,b,lambda_l,lambda_r,lambda_b,n,k):
	"""
		This is a Quadratic in V and can be solved by using 
		sparse linear equations. Quadratic Optimization problem.
	"""
	print('Solving the sparse linear system of equations now... - Theta is a constant; Solving for V')
	A = shape/n + lambda_b*(np.transpose(boundary).dot(boundary)) + lambda_l*line/k
	b = -lambda_b*(np.transpose(boundary).dot(b))
	V = np.linalg.solve(A,b)
	return V

from math import cos, sin

def fix_v_solve_theta(U,lines,thetas,V,rotation_angle,Pk,dx,dy,N,x,y,sdelta):
    k = len(lines)
    delta = 6.1
    beta_min = 1
    beta_max = 10000
    beta = beta_min
    M = len(thetas)
    A = np.zeros((k,M))
    for i in range(len(lines)):
        temp=  lines[i][5]
        A[i][int(temp)-1] = 1
    T_1 = np.zeros((M,M))
    T_1[0:M-1,1:M] = np.eye(M-1)
    T_1[M-1,0] = 1
    D = np.eye(M,M) - T_1
    Pk = computepk(lines,dx,dy,N,x,y)
    Vx = np.zeros(N)
    Vy = np.zeros(N)
    for i in range(N):
        #print(i)
        Vx[i] = V_new[2*i]
        Vy[i] = V_new[2*i+1]
        #print(2*i+1)        
    e_x = Pk.dot(Vx)
    e_y = Pk.dot(Vy)
    e = np.array([e_x,e_y,np.arccos(np.divide(e_x,(np.sqrt(np.square(e_x)+np.square(e_y)))))*180/np.pi])
    e = e.transpose()
    phi_k = np.zeros(k)
    phi = np.zeros((k,100))

    while(beta<=beta_max):

        """
            Fix Theta, update phi. This is a single variable equation
            and is solved by constant lookups, though gradient descent 
            can be used.
        """
        for i in range(len(lines)):
            ek_uk = e[i][2] - lines[i][4]
            bin_ = lines[i][5]
            thetam_k = thetas[int(bin_)-1]
            #del_ = (thetam_k - ek_uk)/100 # This is split as 99 in the original implementation -- check it out . ----
            phi[i] = (np.linspace(ek_uk, thetam_k, num= 100))
            temp = np.zeros(100)
            U = UK[i]     
            for j in range(100):
                Rk = np.array([[cos(phi[i][j]/np.pi*180),-sin(phi[i][j]/np.pi*180)],
                    [sin(phi[i][j]/np.pi*180), cos(phi[i][j]/np.pi*180)]])

                current = (Rk.dot(U).dot(np.transpose(Rk)) - np.eye(2)).dot((e[i][0:2]))

                temp[j] = lambda_l/k*(sum(np.square(current))) + beta*(np.square(phi[i][j] - thetam_k)).sum()

            index = np.argmin(temp)

            phi_k[i] = phi[i][index]
        """
            Fix phi, update theta. Quadratic in Theta and can be
            solved by using linear system of equations.
        """
        H = beta*(np.transpose(A).dot(A)) + lambda_r*np.diag(sdelta) + lambda_r*np.transpose(D).dot(D)
        h = beta*np.transpose(A).dot(phi_k) + (lambda_r*np.diag(sdelta).dot(np.ones(M))*delta)
        thetas = np.linalg.solve(H,h)
        beta = beta*10
    return thetas


In [179]:
t = fix_v_solve_theta(U,lines,thetas,V_new,delta,Pk,dx,dy,N,x,y,sdelta)

TypeError: only size-1 arrays can be converted to Python scalars

In [447]:
delta

inf

In [506]:
from math import cos, sin

def fix_v_solve_theta(U,lines,thetas,V,rotation_angle,Pk,dx,dy,N,x,y,sdelta):
    k = len(lines)
    delta = 6.1
    beta_min = 1
    beta_max = 10000
    beta = beta_min
    M = len(thetas)
    A = np.zeros((k,M))
    for i in range(len(lines)):
        temp=  lines[i][5]
        A[i][int(temp)-1] = 1
    T_1 = np.zeros((M,M))
    T_1[0:M-1,1:M] = np.eye(M-1)
    T_1[M-1,0] = 1
    D = np.eye(M,M) - T_1
    Pk = computepk(lines,dx,dy,N,x,y)
    Vx = np.zeros(N)
    Vy = np.zeros(N)
    for i in range(N):
        #print(i)
        Vx[i] = V_new[2*i]
        Vy[i] = V_new[2*i+1]
        #print(2*i+1)        
    e_x = Pk.dot(Vx)
    e_y = Pk.dot(Vy)
    e = np.array([e_x,e_y,np.arccos(np.divide(e_x,(np.sqrt(np.square(e_x)+np.square(e_y)))))*180/np.pi])
    e = e.transpose()
    phi_k = np.zeros(k)
    phi = np.zeros((k,100))

    while(beta<=beta_max):

        """
            Fix Theta, update phi. This is a single variable equation
            and is solved by constant lookups, though gradient descent 
            can be used.
        """
        for i in range(len(lines)):
            ek_uk = e[i][2] - lines[i][4]
            bin_ = lines[i][5]
            thetam_k = thetas[int(bin_)-1]
            #del_ = (thetam_k - ek_uk)/100 # This is split as 99 in the original implementation -- check it out . ----
            phi[i] = (np.linspace(ek_uk, thetam_k, num= 100))
            temp = np.zeros(100)
            U = UK[i]     
            for j in range(100):
                Rk = np.array([[cos(phi[i][j]/np.pi*180),-sin(phi[i][j]/np.pi*180)],
                    [sin(phi[i][j]/np.pi*180), cos(phi[i][j]/np.pi*180)]])

                current = (Rk.dot(U).dot(np.transpose(Rk)) - np.eye(2)).dot((e[i][0:2]))

                temp[j] = lambda_l/k*(sum(np.square(current))) + beta*(np.square(phi[i][j] - thetam_k)).sum()

            index = np.argmin(temp)

            phi_k[i] = phi[i][index]
        """
            Fix phi, update theta. Quadratic in Theta and can be
            solved by using linear system of equations.
        """
        H = beta*(np.transpose(A).dot(A)) + lambda_r*np.diag(sdelta) + lambda_r*np.transpose(D).dot(D)
        h = beta*np.transpose(A).dot(phi_k) + (lambda_r*np.diag(sdelta).dot(np.ones(M))*delta)
        thetas = np.linalg.solve(H,h)
        beta = beta*10
    return thetas

In [501]:
thetas

array([6.09175018, 5.88202441, 5.74777008, 5.86379645, 5.89325734,
       5.95013548, 5.92670099, 5.92689194, 5.9651599 , 6.00342786,
       5.99812476, 5.98076273, 5.98030067, 6.00669636, 6.0103521 ,
       5.99129603, 5.98553905, 5.97978207, 5.97829181, 5.97882864,
       5.98121562, 5.99092423, 6.02438409, 6.02936747, 5.98861575,
       5.97931276, 5.97845348, 5.97759421, 5.94394384, 5.9743511 ,
       5.97846104, 5.9782964 , 5.98110327, 5.98391014, 5.9990148 ,
       6.0265037 , 6.03450049, 6.03555199, 6.04581643, 6.05951174,
       6.06061003, 6.03863231, 6.03898216, 6.0423008 , 6.09190424,
       6.09046205, 6.00680541, 6.00095456, 6.04021714, 6.04541075,
       6.07322883, 6.07405536, 6.03587174, 6.08770539, 6.07511542,
       6.0355935 , 6.00740968, 5.97922585, 5.97884359, 5.98119066,
       5.97858832, 5.9761718 , 5.97847237, 5.98077294, 5.98629615,
       5.97898564, 5.98321197, 6.00768459, 5.99327497, 5.97886535,
       5.97844733, 5.97896989, 5.97949244, 6.00795424, 6.03641

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1.])

In [494]:
delta

inf

In [492]:
np.diag(sdelta).dot(np.ones(M))*

/home/deepak/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in multiply
  """Entry point for launching an IPython kernel.


array([inf, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, inf, inf, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, inf])

In [415]:
(lambda_r*np.diag(sdelta).dot(np.ones(M))*delta)

/home/deepak/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in multiply
  """Entry point for launching an IPython kernel.


array([inf, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, inf, inf, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, inf])

In [503]:
	
def fix_v_solve_theta(U,lines,thetas,V,rotation_angle,Pk,dx,dy,N,x,y,sdelta):
	"""
		This is a non trivial function to solve. Use the half
		quadratic method to solve this equation.
	"""
	k = len(lines)
	delta = rotation_angle
	beta_min = 1
	beta_max = 10000
	beta = beta_min
	M = len(thetas)
	A = np.zeros((k,M))
	for i in range(len(lines)):
		temp=  lines[i][5]
		A[i][int(temp)-1] = 1
	T_1 = np.zeros((M,M))
	T_1[0:M-1,1:M] = np.eye(M-1)
	T_1[M-1,0] = 1
	D = np.eye(M,M) - T_1
	Pk = computepk(lines,dx,dy,N,x,y)
	Vx = np.zeros(N)
	Vy = np.zeros(N)
	for i in range(N):
		#print(i)
		Vx = V[2*i]
		Vy = V[2*i+1]
		#print(2*i+1)        
	e_x = Pk.dot(Vx)
	e_y = Pk.dot(Vy)
	e = np.array([e_x,e_y,np.arccos(e_x/math.sqrt(e_x**2 + e_y**2))/pi*180])
	phi_k = np.zeros(k)
	phi = zeros(k,100)

	while(beta<=beta_max):

		"""
			Fix Theta, update phi. This is a single variable equation
			and is solved by constant lookups, though gradient descent 
			can be used.
		"""
		for i in range(len(lines)):
			ek_uk = e[i][2] - lines[i][4]
			bin_ = lines[i][5]
			thetam_k = thetas(bin_)
			phi[i] = np.linspace(ek_uk,thetam_k,((thetam_k-ek_uk)/100))
			temp = np.zeros(100)
			U = UK[i]     
			for j in range(100):
				Rk = np.array([cos(phi[i][j]/np.pi*180),-sin(phi[i][j]/np.pi*180)],
					[sin(phi[i][j]/np.pi*180), cos(phi[i][j]/np.pi*180)])

				current = (Rk.dot(U).dot(np.transpose(Rk)) - np.eye(2)).dot((e[i][0:2]))
				temp[j] = lambda_l/k*(np.linalg.norm(current)**2) + beta*np.linalg.norm(phi[i][j] - thetam_k)**2
			index = min(temp)
			phi_k[i] = index
		"""
			Fix phi, update theta. Quadratic in Theta and can be
			solved by using linear system of equations.
		"""
		H = beta*(np.transpose(A).dot(A)) + lambda_r*np.diag(sdelta) + lambda_r*np.transpose(D).dot(D)
		h = beta*np.transpose(A).dot(phi_k) + lambda_r*diag(sdelta)*np.ones((M,1))*delta
		thetas = np.linalg.solve(H,h)
		
		beta = beta*10

	return thetas 

IndentationError: expected an indented block (<ipython-input-503-e28e6f5ebdbd>, line 6)

In [421]:
beta*np.transpose(A).dot(phi_k)

array([3.94303064e+01, 5.83377243e+00, 1.85072749e+00, 3.86989672e+00,
       7.64893906e-01, 6.21191640e-01, 3.15356502e-01, 1.07813438e-01,
       0.00000000e+00, 9.70523997e-01, 7.55391111e-01, 1.28983248e+00,
       1.83438265e+00, 5.16684196e-01, 1.72479364e-01, 1.90122773e-01,
       0.00000000e+00, 2.88143370e-01, 1.04595162e-01, 2.20390819e-01,
       1.24258235e-02, 1.00230685e-01, 1.46235969e-01, 2.79349987e-01,
       4.03328268e-01, 1.88204346e-01, 0.00000000e+00, 2.76291098e-02,
       1.09900580e+00, 1.61015646e-01, 7.12312459e-01, 6.26985915e-02,
       0.00000000e+00, 3.28135561e-01, 1.22053533e-01, 2.47260762e-02,
       2.50527489e-02, 6.61710476e-01, 6.94266799e-01, 3.98491955e-01,
       1.34198456e+00, 2.52097313e-01, 3.33728238e+00, 7.37044350e+00,
       3.96190612e+01, 3.86575606e+01, 1.10450686e+01, 4.21524395e+00,
       3.58040817e+00, 1.44193759e+00, 4.61506806e-01, 1.35672308e+00,
       8.87345551e-02, 5.85940799e-01, 2.19997232e-01, 1.75934819e-01,
      

In [383]:
 + lambda_r*np.diag(sdelta)*np.ones((M,1))*delta).shape

(90, 90)

In [386]:
(lambda_r*np.diag(sdelta)*np.ones((M))*delta).shape, (beta*np.transpose(A).dot(phi_k)).shape

((90, 90), (90,))

In [393]:
(lambda_r*np.diag(sdelta).dot(np.ones(M))*delta)

(90,)

In [341]:
len(np.linspace(1,1,100));

np.linspace(ek_uk,del_,thetam_k)

/home/deepak/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: object of type <class 'numpy.float64'> cannot be safely interpreted as an integer.
  This is separate from the ipykernel package so we can avoid doing imports until


array([5.1818592 , 4.14732364, 3.11278808, 2.07825252, 1.04371697,
       0.00918141])

In [350]:
len(np.arange(ek_uk, thetam_k,del_))

101

In [347]:
len(np.arange(ek_uk, thetam_k,del_))

100

In [335]:
e.transpose()

array([[-7.69532617e+00, -1.28528853e+01,  4.33211302e+00, ...,
        -1.37866393e+01, -1.42289843e+01, -1.19928707e+01],
       [-5.30913905e-02, -1.29970251e-01,  1.29188703e+01, ...,
        -1.27585639e+00, -5.79752009e-01, -2.17269792e-01],
       [ 1.79604713e+02,  1.79420636e+02,  7.14620398e+01, ...,
         1.74712738e+02,  1.77666806e+02,  1.78962110e+02]])

In [306]:
np.pi == math.pi

True

In [299]:
np.divide(np.array([1,22,3]),np.array([2,22,4]))

array([0.5 , 1.  , 0.75])

In [291]:
e.transpose()[:10]

array([[-7.69532617e+00, -5.30913905e-02,  9.06683677e+01],
       [-1.28528853e+01, -1.29970251e-01,  9.11163661e+01],
       [ 4.33211302e+00,  1.29188703e+01,  8.96237457e+01],
       [ 1.81689385e+00,  5.46911734e+00,  8.98421994e+01],
       [ 8.21059628e+00,  1.92669870e+00,  8.92868770e+01],
       [ 7.07740679e+00,  1.63934377e+00,  8.93853031e+01],
       [ 7.69067439e+00,  7.45352037e-01,  8.93320364e+01],
       [ 2.93438247e+01,  3.10633140e+00,  8.74505965e+01],
       [-1.87047775e+01,  7.65119366e-01,  9.16247603e+01],
       [-9.59401636e+00,  4.06111111e-01,  9.08332864e+01]])

In [215]:
np.linspace(ek_uk,thetam_k[0],((thetam_k[0]-ek_uk)/100))

/home/deepak/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: object of type <class 'numpy.float64'> cannot be safely interpreted as an integer.
  """Entry point for launching an IPython kernel.


array([], dtype=float64)

In [219]:
np.lina

array([ 0., 10.])

In [209]:
ek_uk, thetam_k

(23.635810794815853, array([6.1]))

In [136]:
import numpy as np
import math
import pandas as pd

"""
	Incomplete as on 10th November! Bad - This is horrible!
"""

def getuk(lines):
	uk = np.zeros((len(lines),2))
	for i in range(len(lines)):
		x =  lines[i][2] - lines[i][0]
		y = lines[i][3] - lines[i][1]
		temp = np.array([x,y])
		try:
			temp = temp/np.linalg.norm(temp)
			uk[i] = temp
		except:
			temp = np.array([0,0])
			uk[i] = temp
	return uk

dx = x_len;dy = y_len; x = vertexX; y =vertexY;

U_final = np.zeros((len(lines),2,2))
"""
    This is to form the Energy Function for 
    adding the constraint to preserve the line segments.
    This is a very important part of the project.
"""
#UAll = np.zeros(((2,2),len(lines)))
uk = getuk(lines)
uk = np.matrix(uk)
N = number_of_vertices
Pk = computepk(lines,dx,dy,N,x,y)

matrix = np.zeros((2*N,2*N))
tempPk1 = np.zeros((2*N))
tempPk2 = np.zeros((2*N))
Pk_ = np.zeros(((2*len(lines)),2*N))

for i in range(len(lines)):
    Pk_final = np.zeros((2,2*N))
    for j in range(N):
        tempPk1[2*j] = Pk[i][j]
        tempPk2[2*j+1] = Pk[i][j]
    Pk_final[0] = tempPk1
    Pk_final[1] = tempPk2
    theta = thetas[int(lines[i][5])]*np.pi/180
    U = uk[i].transpose().dot(uk[i])
    val = (np.array(uk[i]).dot(np.transpose(np.array(uk[i]))))
    U = np.array(U)*val
    U_final[i] = U
    Rk = np.array([[math.cos(theta),-math.sin(theta)],[math.sin(theta),math.cos(theta)]])
    temp = (Rk.dot(U).dot(np.transpose(Rk)) - np.eye(2)).dot(Pk_final)
    inter = np.transpose(temp).dot(temp)
    matrix+=inter
    print(np.count_nonzero(temp))
    df = pd.DataFrame(temp)
    df.to_csv('temp.csv')
    Pk_[2*(i)] = Pk_final[0]
    Pk_[2*i+1] = Pk_final[1]
    break
    #print(U_final[i])

16


In [137]:
Pk.shape

(1313, 690)

In [138]:
len(lines)

1313

In [139]:
lines[i]

array([553.183267  , 488.74241   , 545.63736928, 489.45454545,
       174.6087442 ,   1.        ])

In [140]:
Rk

array([[ 0.99433794, -0.10626407],
       [ 0.10626407,  0.99433794]])

In [141]:
U

array([[ 0.9911722 , -0.09354074],
       [-0.09354074,  0.0088278 ]])

In [142]:
Pk_final

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [146]:
[np.nonzero(Pk[0])]

[(array([406, 407, 429, 430]),)]

In [119]:
tempPk1


array([0., 0., 0., ..., 0., 0., 0.])

In [120]:

np.nonzero(tempPk1)

(array([], dtype=int64),)

In [121]:
np.nonzero(tempPk2)

(array([], dtype=int64),)

In [23]:
uk[i].transpose()

matrix([[-0.99615193],
        [ 0.08764323]])

In [27]:
uk[len(lines)-10:]

matrix([[-1.        ,  0.        ],
        [ 1.        ,  0.        ],
        [-0.85219859,  0.52321847],
        [-0.76140239,  0.64827957],
        [ 0.08413491,  0.99645437],
        [ 0.99394827,  0.10984911],
        [-0.99397914,  0.10956949],
        [-0.99994606,  0.01038659],
        [-0.99834078,  0.05758204],
        [-0.99615193,  0.08764323]])

In [28]:
lines[-1]

array([538.073202  , 497.536265  , 526.4137931 , 498.56208072,
       174.97196151,   1.        ])

In [33]:
lines[len(lines)-10:]

array([[688.125     , 529.375     , 668.125     , 529.375     ,
        180.        ,   4.        ],
       [668.125     , 529.375     , 681.24137931, 529.375     ,
        180.        ,   4.        ],
       [ 85.3211141 , 183.54545455,  68.003823  , 194.177632  ,
        148.45161106,  78.        ],
       [ 13.731523  , 286.33736   ,   4.956842  , 293.808372  ,
        139.58798611,  73.        ],
       [423.436175  , 463.098726  , 425.025808  , 481.925592  ,
         85.1737191 ,  46.        ],
       [217.88902   , 146.510219  , 234.352375  , 148.329715  ,
          6.30661755,   7.        ],
       [354.429462  , 396.119066  , 341.807256  , 397.510452  ,
        173.70950092,  90.        ],
       [650.27586207, 531.88866577, 636.876588  , 532.027846  ,
        179.4048817 ,   3.        ],
       [831.90548   , 244.903449  , 818.129212  , 245.698033  ,
        176.69896624,   2.        ],
       [538.073202  , 497.536265  , 526.4137931 , 498.56208072,
        174.97196151,   1. 

In [34]:
len(uk)==len(lines)

True

In [35]:
uk[len(uk)-1]

matrix([[-0.99615193,  0.08764323]])

In [36]:
lines[len(lines)-1]

array([538.073202  , 497.536265  , 526.4137931 , 498.56208072,
       174.97196151,   1.        ])

In [39]:
uk = np.zeros((len(lines),2))
for i in range(len(lines)):
    if i==len(lines)-1:
        x =  lines[i][2] - lines[i][0]
        y = lines[i][3] - lines[i][1]
        temp = np.array([x,y])
        print(x,y)
        try:
            temp = temp/np.linalg.norm(temp)
            uk[i] = temp
        except:
            temp = np.array([0,0])
            uk[i] = temp

-11.659408896551781 1.0258157163588635


In [43]:
uk[i] = uk[i]/np.linalg.norm(uk[i])

In [44]:
uk[i]

array([-0.99615193,  0.08764323])

In [45]:
1.0258/0.99999


1.025810258102581